# Import modules and dependencies

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from api_keys import api_keys
from os.path import exists
import requests
import gmaps
pd.set_option('display.max_columns', None)

## Generate the .csv file with the data from the requests

In [11]:
if not exists("Resources/Hotel_Data.csv"):
    keys = api_keys()

    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    # location=-33.8670522,151.1957362&    #(lat, long)#
    # radius=5000&
    # type=lodging&
    # keyword=hotel&
    # key=keys.get_google thinhhhe

    raw_data = pd.read_csv("Resources/Weather_Data.csv")

    # Filter the data to only get cities with 60-80 degrees, wind speed less than 10, 0 cloudiness
    data = raw_data[(raw_data["Temperature (Fahrenheit)"] >= 60) & (raw_data["Temperature (Fahrenheit)"] <= 80)]
    data = data[data["Wind Speed (mph)"] <= 10]
    data = data[data["Cloudiness"] == 0]
    plt.scatter(raw_data["Longitude"], raw_data["Latitude"])
    # plt.show() Todo undo comment here
    print(data.head(26))
    hotel_name = []
    hotel_city = []
    hotel_country = []
    bad_cities = []
    for x in range(len(data)):
        try:
            lat = data["Latitude"].tolist()[x]
            long = data["Longitude"].tolist()[x]
            request = requests.get(
                f'{url}location={lat},{long}&radius=5000&type=lodging&keyword=hotel&key={keys.get_google_key()}'
            ).json()
            hotel_country.append(request["results"][0]["plus_code"]["compound_code"].split(",")[1])
            hotel_city.append(data["City"].tolist()[x])
            hotel_name.append(request["results"][0]["name"])
        except Exception as e:
            print(f'{x} index gives Foogle an issue! Will not be adding in a hotel for {data["City"].tolist()[x]}')
            bad_cities.append(x)

    print(f'Hotel_Name length = {len(hotel_name)}')
    print(f'Hotel_City length = {len(hotel_city)}')
    print(f'Hotel_Country length = {len(hotel_country)}')

    hotel_df = pd.DataFrame({
        "Hotel Name": hotel_name,
        "Hotel City": hotel_city,
        "Hotel Country": hotel_country
    })
    hotel_df.to_csv("Resources/Hotel_Data.csv", index=False)


## Read the data into DataFrames

In [12]:
weather_df = pd.read_csv("Resources/Weather_Data.csv")
vacation_df = pd.read_csv("Resources/Hotel_Data.csv")
print(weather_df.columns)
print(vacation_df.columns)

Index(['City', 'Longitude', 'Latitude', 'Temperature (Fahrenheit)', 'Humidity',
       'Cloudiness', 'Wind Speed (mph)'],
      dtype='object')
Index(['Hotel Name', 'Hotel City', 'Hotel Country'], dtype='object')


## Customize and create the figure

In [15]:
figure_layout = {
    'width': '900px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

### Create the global heatmap

In [16]:
lat_long = weather_df[["Latitude","Longitude"]]
humidity = weather_df["Humidity"]
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create the markers for the ideal vacation locations (Assuming I could travel...)

In [43]:
vacation_df["Hotel Latitude"] = [
    weather_df[weather_df["City"] == vacation_df["Hotel City"].tolist()[x]]["Latitude"].sum()
    for x in range(len(vacation_df))
]
vacation_df["Hotel Longitude"] = [
    weather_df[weather_df["City"] == vacation_df["Hotel City"].tolist()[x]]["Longitude"].sum()
    for x in range(len(vacation_df))
]
vacation_df.head()

,Hotel Name,Hotel City,Hotel Country,Hotel Latitude,Hotel Longitude
0,The Glen Boutique Hotel & Spa,Cape Town,South Africa,-33.9258,18.4232
1,Herms Restaurant and Boutique Hotel,Port Elizabeth,South Africa,-33.9180,25.5701
2,Hotel Chiqui Santander,Santander,Spain,43.4647,-3.8044
3,Hosteria Sampedro,Rawson,Chubut Province,-43.3002,-65.1023
4,Гостиница Respect,Balkhash,Kazakhstan,46.8440,74.9804


In [47]:
hotel_longlat = vacation_df[["Hotel Latitude", "Hotel Longitude"]]
markers = gmaps.marker_layer(hotel_longlat,
                            info_box_content =[
                                f'{vacation_df["Hotel Name"].tolist()[x]}, {vacation_df["Hotel City"].tolist()[x]}, {vacation_df["Hotel Country"].tolist()[x]}'
                                for x in range(len(vacation_df))
                            ])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…